In [1]:
!rm -rf missing_symbols
!git clone https://github.com/marco-c/missing_symbols.git
dbutils.library.installPyPI("google-cloud-bigquery", "1.20.0")
dbutils.library.restartPython()

In [2]:
import os
from datetime import datetime, timedelta
from pyspark.sql import functions
import boto3
from urllib.parse import urljoin
import requests

In [3]:
known_modules = set([module[:-4].lower() for module in os.listdir('missing_symbols/known_modules')])

In [4]:
dataset = (spark.read.format("bigquery")
    .option("table", "moz-fx-data-derived-datasets.telemetry_derived.socorro_crash_v2")
    .load()
    .where("crash_date >= to_date('{}')".format((datetime.utcnow() - timedelta(3)).strftime('%Y-%m-%d')))
)

In [5]:
# This is a comparison of results if we use a BigQuery query for all the logic rather than Spark.

'''query = """
WITH
  base AS (
  SELECT
    uuid,
    json_dump.modules.list AS modules,
    ROW_NUMBER() OVER (PARTITION BY uuid) AS _n
  FROM
    `moz-fx-data-derived-datasets.telemetry_derived.socorro_crash_v2`
  WHERE
    crash_date >= "2019-12-06"
    AND product = 'Firefox' ),
  deduped AS (
  SELECT
    * EXCEPT(_n)
  FROM
    base
  WHERE
    _n = 1 ),
  exploded AS (
  SELECT
    module.element.*
  FROM
    deduped
  CROSS JOIN
    UNNEST(modules) AS module )
SELECT
  filename,
  version,
  debug_id,
  debug_file,
  COUNT(*) AS n
FROM
  exploded
WHERE
  missing_symbols
  AND filename NOT LIKE '%(deleted)%'
  AND LOWER(filename) NOT IN {known_modules}
GROUP BY
  1,
  2,
  3,
  4
""".format(known_modules=tuple(known_modules))

from google.cloud import bigquery

bq = bigquery.Client()

dataset_id = 'analysis'
table_ref = bq.dataset(dataset_id, project='moz-fx-data-shared-prod').table('klukas_modules_with_missing_symbols')
job_config = bigquery.QueryJobConfig()
job_config.destination = table_ref
job_config.write_disposition = 'WRITE_TRUNCATE'

query_job = bq.query(query, job_config=job_config)
query_job.result()

df = spark.read.format('bigquery') \
    .option('project', 'moz-fx-data-shared-prod') \
    .option('dataset', query_job.destination.dataset_id) \
    .option('table', query_job.destination.table_id) \
    .load()
modules_via_query = df.rdd\
  .flatMap(lambda m: [((m['filename'], (m['version'], m['debug_id'], m['debug_file'])), m['n'])])\
  .reduceByKey(lambda x, y: x + y)\
  .map(lambda v: (v[0][0], [(v[0][1], v[1])]))\
  .reduceByKey(lambda x, y: x + y)\
  .sortBy(lambda v: sum(count for ver,count in v[1]), ascending=False)\
  .collect()'''

In [6]:
modules = dataset\
.filter(dataset['product'] == 'Firefox')\
.select(['uuid'] + [functions.explode((dataset['json_dump']['modules']['list'])).alias('module')])\
.dropDuplicates(['uuid', 'module'])\
.select(['module'])\
.rdd\
.map(lambda v: v['module']['element'])\
.filter(lambda m: m['missing_symbols'] and m['filename'].lower() not in known_modules and '(deleted)' not in m['filename'])\
.flatMap(lambda m: [((m['filename'], (m['version'], m['debug_id'], m['debug_file'])), 1)])\
.reduceByKey(lambda x, y: x + y)\
.map(lambda v: (v[0][0], [(v[0][1], v[1])]))\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda v: sum(count for ver,count in v[1]), ascending=False)\
.collect()


In [7]:
len(modules)

In [8]:
[(module, sum(count for ver, count in versions)) for module, versions in modules[:50]]

In [9]:
top_missing = sorted([(name, version, count) for name, versions in modules for version, count in versions if count > 2000], key=lambda m: m[2], reverse=True)

In [10]:
len(top_missing)

In [11]:
with open('missing_symbols/firefox_modules.txt', 'r') as f:
    firefox_modules = [m.lower() for m in f.read().split('\n') if m.strip() != '']

In [12]:
with open('missing_symbols/windows_modules.txt', 'r') as f:
    windows_modules = [m.lower() for m in f.read().split('\n') if m.strip() != '']

In [13]:
r = requests.get('https://product-details.mozilla.org/1.0/firefox_history_major_releases.json')
firefox_versions = r.json()
old_firefox_versions = []
for version, date in firefox_versions.items():
    delta = datetime.utcnow() - datetime.strptime(date, '%Y-%m-%d')
    if abs(delta.days) > 730:
        old_firefox_versions.append(version[:version.index('.')])

def is_old_firefox_module(module_info):
    name, (version, _, _), _ = module_info
    if name.lower() not in firefox_modules:
        return False

    return any(version.startswith(v + '.') for v in old_firefox_versions)

top_missing = [m for m in top_missing if not is_old_firefox_module(m)]

In [14]:
def are_symbols_available(debug_file, debug_id):
    if not debug_file or not debug_id:
        return False

    url = urljoin('https://symbols.mozilla.org/', '{}/{}/{}'.format(debug_file, debug_id, debug_file if not debug_file.endswith('.pdb') else debug_file[:-3] + 'sym'))
    r = requests.head(url)
    return r.ok

top_missing_with_avail_info = [(name, version, debug_id, count, are_symbols_available(debug_id, debug_file)) for name, (version, debug_id, debug_file), count in top_missing]

In [15]:
subject = 'Weekly report of modules with missing symbols in crash reports'

body = """
<table style="border-collapse:collapse;">
  <tr>
  <th style="border: 1px solid black;">Name</th>
  <th style="border: 1px solid black;">Version</th>
  <th style="border: 1px solid black;">Debug ID</th>
  <th style="border: 1px solid black;"># of crash reports</th>
</tr>
"""
any_available = False
for name, version, debug_id, count, are_available_now in top_missing_with_avail_info:
    body += '<tr>'
    body += '<td style="border: 1px solid black;">'
    if name.lower() in firefox_modules:
        if debug_id:
            body += '<span style="color:red;">%s</span>' % name
        else:
            body += '<span style="color:orange;">%s</span>' % name
    elif name.lower() in windows_modules:
        body += '<span style="color:blue;">%s</span>' % name
    else:
        body += name
    if are_available_now:
        body += ' (*)'
        any_available = True
    body += '</td>'
    body += '<td style="border: 1px solid black;">%s</td>' % version
    body += '<td style="border: 1px solid black;">%s</td>' % debug_id
    body += '<td style="border: 1px solid black;">%d</td>' % count
    body += '</tr>'
body += '</table>'

body += '<pre>'

if any_available:
    body += """
(*) We now have symbols for the modules marked with an asterisk. We could
reprocess them to improve stack traces (and maybe signatures) of some crash reports.\n
"""

body += """
The number of crash reports refers to the past 3 days.
Only modules with at least 2,000 crash reports are shown in this list.

Firefox own modules, for which we should have symbols, and have the debug ID are colored in red.
For Firefox own modules, where we don't have a debug ID are colored in orange.
OS modules, for which we should have symbols, are colored in blue.

If you see modules that shouldn't be in this list as it's expected not
to have their symbols, either contact mcastelluccio@mozilla.com or open
a PR to add them to https://github.com/marco-c/missing_symbols/tree/master/known_modules.
"""

body += '</pre>'

client = boto3.client(
  "ses",
  aws_access_key_id=dbutils.secrets.get("data-email", "access_key_id"),
  aws_secret_access_key=dbutils.secrets.get("data-email", "secret_access_key"),
  region_name="us-west-2",
)

client.send_email(
    Source='mcastelluccio@data.mozaws.net',
    Destination={
        'ToAddresses': ['mcastelluccio@mozilla.com', 'release-mgmt@mozilla.com', 'stability@mozilla.org'],
        'CcAddresses': [],
    },
    Message={
        'Subject': {'Data': subject, 'Charset': 'UTF-8'},
        'Body': {'Html': {'Data': body, 'Charset': 'UTF-8'}}
    }
)['MessageId']